In [2]:
import numpy as np
import pandas as pd

In [16]:
data = pd.read_csv('data/allegations.csv', index_col=0)

In [17]:
data.head()

,unique_mos_id,shield_no,complaint_id,mos_ethnicity,mos_gender,mos_age_incident,complainant_ethnicity,complainant_gender,complainant_age_incident,fado_type,allegation,precinct,contact_reason,Substantiated
0,10004,8409.0,42835,Hispanic,M,32,Black,Female,38.0,Abuse of Authority,Failure to provide RTKA card,78.0,Report-domestic dispute,True
1,10007,5952.0,24601,White,M,24,Black,Male,26.0,Discourtesy,Action,67.0,Moving violation,True
2,10007,5952.0,24601,White,M,24,Black,Male,26.0,Offensive Language,Race,67.0,Moving violation,True
3,10007,5952.0,26146,White,M,25,Black,Male,45.0,Abuse of Authority,Question,67.0,PD suspected C/V of violation/crime - street,True
4,10009,24058.0,40253,Hispanic,F,39,NaN,NaN,16.0,Force,Physical force,67.0,Report-dispute,True


In [18]:
data.isna().sum()

unique_mos_id                  0
shield_no                   5392
complaint_id                   0
mos_ethnicity                  0
mos_gender                     0
mos_age_incident               0
complainant_ethnicity       5505
complainant_gender          4195
complainant_age_incident    4829
fado_type                      0
allegation                     1
precinct                      48
contact_reason               199
Substantiated                  0
dtype: int64

In [28]:
relevant = data.rename({'Substantiated': 'substantiated'}, axis = 1)

In [30]:
relevant = relevant[[
    'complainant_ethnicity', 'complainant_gender', 'complainant_age_incident', 'allegation', 'contact_reason', 'substantiated'
]]

In [31]:
relevant.head(10)

,complainant_ethnicity,complainant_gender,complainant_age_incident,allegation,contact_reason,substantiated
0,Black,Female,38.0,Failure to provide RTKA card,Report-domestic dispute,True
1,Black,Male,26.0,Action,Moving violation,True
2,Black,Male,26.0,Race,Moving violation,True
3,Black,Male,45.0,Question,PD suspected C/V of violation/crime - street,True
4,NaN,NaN,16.0,Physical force,Report-dispute,True
5,White,Male,31.0,Refusal to process civilian complaint,C/V telephoned PCT,True
6,White,Male,34.0,Sexual orientation,PD suspected C/V of violation/crime - street,True
7,Asian,Male,60.0,Word,Moving violation,False
8,Asian,Male,39.0,Refusal to provide shield number,Moving violation,False
9,Asian,Male,39.0,Retaliatory summons,Moving violation,True


In [32]:
relevant[['complainant_ethnicity', 'substantiated']].groupby("complainant_ethnicity").mean()

,substantiated
complainant_ethnicity,
American Indian,0.359375
Asian,0.291353
Black,0.237992
Hispanic,0.244707
Other Race,0.268833
Refused,0.270270
White,0.279195


In [42]:
def mcar(dataset : pd.DataFrame, n=10000):
    """
    Missing Completely at Random -- there is no dependency of the rate of
    missingness on other values or the value itself.

    Generated by simple sampling.
    """
    sampled = dataset.sample(n, replace=True)
    return sampled

    # don't we also have to set the outcomes to be split 50/50 between True and False?

In [45]:
def mar(dataset : pd.DataFrame, n=10000):
    """
    Missing at Random -- data is missing due to dependency on another
    variable with no missing values.

    Generated by calculating the probability the data is missing given outcome.
    Pandas probably has an easy way to do this.
    """
    pass

In [ ]:
def nmar(dataset : pd.DataFrame, n=10000):
    """
    Not Missing at Random -- data is missing due to the value of the variable
    itself. For example, sales not recorded for the stores that produced less
    than a specific amount of revenue.

    Generated by fancy math that I don't remember. Alec save me please.
    """
    pass